#### This is the Jupiter Notebook for my Tidy Data Project, using the data file regarding the federal research and development budgets across different government departments.

In tyding the data, I must first define my dataframe

In [19]:
import pandas as pd #import pandas dictionary to be referred to later

df_fedrd = pd.read_csv('fed_rd_year&gdp.csv') # define dataframe

Next, I need to melt my dataframe such that the different variables are defined. This step is what reorients the columns such that the years and GDPs, which are values, are not correctly different rows. It must still be split such that the year and GDP variables are different columns. 

In [ ]:
df_fedrd_melted = pd.melt(df_fedrd,
                          id_vars = 'department', # holds constant the column for department
                          value_vars = df_fedrd.columns[1:], # selects all the columns to be reoriented
                          var_name = 'Year_GDP', # renames the column now containing the previous column names
                          value_name='Research and Development Budget') #names the column containing all of the values from the original untidy dataframe

print(df_fedrd_melted)

    department                  Year_GDP  Research and Development Budget
0          DHS   1976_gdp1790000000000.0                              NaN
1          DOC   1976_gdp1790000000000.0                     8.190000e+08
2          DOD   1976_gdp1790000000000.0                     3.569600e+10
3          DOE   1976_gdp1790000000000.0                     1.088200e+10
4          DOT   1976_gdp1790000000000.0                     1.142000e+09
..         ...                       ...                              ...
583        NIH  2017_gdp19177000000000.0                     3.305200e+10
584        NSF  2017_gdp19177000000000.0                     6.040000e+09
585      Other  2017_gdp19177000000000.0                     1.553000e+09
586       USDA  2017_gdp19177000000000.0                     2.625000e+09
587         VA  2017_gdp19177000000000.0                     1.367000e+09

[588 rows x 3 columns]


Now, I still need to split the dataframe such that the gdp and year are different columns, with correct formatting

In [ ]:
df_fedrd_melted[['Year','GDP']] = df_fedrd_melted['Year_GDP'].str.split('_', expand=True) # splits the column with two variables into two columns, expand=true makes them two columns instead of a list of two variables
df_fedrd_melted_tidy = df_fedrd_melted.drop('Year_GDP', axis=1) # gets rid of messy/unwanted column
df_fedrd_melted_tidy['GDP'] = df_fedrd_melted_tidy['GDP'].str.replace('gdp','') # fixes formatting such that GDP column does not contain unnecessary information
df_fedrd_melted_tidy

,department,Research and Development Budget,Year,GDP
0,DHS,NaN,1976,1790000000000.0
1,DOC,8.190000e+08,1976,1790000000000.0
2,DOD,3.569600e+10,1976,1790000000000.0
3,DOE,1.088200e+10,1976,1790000000000.0
4,DOT,1.142000e+09,1976,1790000000000.0
...,...,...,...,...
583,NIH,3.305200e+10,2017,19177000000000.0
584,NSF,6.040000e+09,2017,19177000000000.0
585,Other,1.553000e+09,2017,19177000000000.0
586,USDA,2.625000e+09,2017,19177000000000.0
